In [6]:
# Import required libraries
from radiant_mlhub import Dataset
from pathlib import Path
import os
from radiant_mlhub import client, get_session
from shapely.geometry import box
import geopandas as gpd
import yalm
import json

import urllib.parse
import re
from pathlib import Path
import itertools as it
from functools import partial
from concurrent.futures import ThreadPoolExecutor

from tqdm.notebook import tqdm

In [2]:
PATH_DATAOUT = '/Users/noeliaotero/Documents/Geo_analysis/data/'

In [3]:
# Open json file
gdf = gpd.read_file('/Users/noeliaotero/Documents/Geo_analysis/cameroon.json')
# Set the crs to EPSG 4326 in case of our data is in aonther crs
gdf  = gdf.to_crs("EPSG:4326")
# Get AOI bounds
bbox_aoi = gdf.geometry.total_bounds
# Get AOI centroid for plotting with folium
centroid_aoi = [box(*bbox_aoi).centroid.x, box(*bbox_aoi).centroid.y]
# I will use this later
print(centroid_aoi)

[12.177475309471191, 5.02154295782988]


In [4]:
os.environ['MLHUB_API_KEY'] = 'e24d1e7931b752bfabe579a9e39077636bbb27f90cec5037297e453c488dca15'

In [5]:

# List of all Datasets
datasets = Dataset.list()
for dataset in datasets:
    print(dataset)

umd_mali_crop_type: 2019 Mali CropType Training Data
idiv_asia_crop_type: A crop type dataset for consistent land cover classification in Central Asia
dlr_fusion_competition_germany: A Fusion Dataset for Crop Type Classification in Germany
ref_fusion_competition_south_africa: A Fusion Dataset for Crop Type Classification in Western Cape, South Africa
ref_agrifieldnet_competition_v1: AgriFieldNet Competition Dataset
bigearthnet_v1: BigEarthNet
microsoft_chesapeake: Chesapeake Land Cover
c2smsfloods_v1: Cloud to Street - Microsoft flood dataset
csu_synthetic_attribution: CSU Synthetic Attribution Benchmark Dataset
ref_african_crops_kenya_02: CV4A Kenya Crop Type Competition
ref_african_crops_uganda_01: Dalberg Data Insights Crop Type Uganda
rti_rwanda_crop_type: Drone Imagery Classification Training Dataset for Crop Types in Rwanda
cgiar_east_africa_agricultural_field_centers: East Africa Agricultural Field Centers
lacuna_fund_eotg_v1: Eyes on the Ground Image Data
ref_african_crops_tanz

In [81]:
dataset = Dataset.fetch('ref_landcovernet_af_v1')

print(f'Title: {dataset.title}')
print(f'DOI: {dataset.doi}')
print(f'Citation: {dataset.citation}')
print('\nCollection IDs and License:')
for collection in dataset.collections:
    print(f'    {collection.id} : {collection.license}')

Title: LandCoverNet Africa
DOI: 10.34911/rdnt.d2ce8i
Citation: Alemohammad S.H., Ballantyne A., Bromberg Gaber Y., Booth K., Nakanuku-Diggs L., & Miglarese A.H. (2020) "LandCoverNet Africa: A Geographically Diverse Land Cover Classification Training Dataset", Version 1.0, Radiant MLHub. [Date Accessed] https://doi.org/10.34911/rdnt.d2ce8i

Collection IDs and License:
    ref_landcovernet_af_v1_source_sentinel_2 : CC-BY-4.0
    ref_landcovernet_af_v1_source_sentinel_1 : CC-BY-4.0
    ref_landcovernet_af_v1_source_landsat_8 : CC-BY-4.0
    ref_landcovernet_af_v1_labels : CC-BY-4.0


In [88]:
bb = """
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              12.177475309471191,
                5.02154295782988
            ],
            [
              12.177475309471191,
             5.02154295782988
            ],
            [
              12.177475309471191,
             5.02154295782988
            ],
            [
              12.177475309471191,
                 5.02154295782988
            ],
            [
              12.177475309471191,
                5.02154295782988
            ]
          ]
        ]
      }
    }
"""

In [82]:
aoi = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
            12.177475309471191,
            5.02154295782988
        ],
        [
            12.177475309471191,
            5.02154295782988
        ],
        [
            12.177475309471191,
            5.02154295782988
        ],
        [
            12.aoi177475309471191,
            5.02154295782988
        ],
        [
            12.177475309471191,
            5.02154295782988
        ]
      ]
    ]
  }
}

In [85]:
from dateutil.parser import parse


my_start_date=parse("2018-02-28T00:00:00+0000")
my_end_date=parse("2018-01-01T00:00:00+0000")


my_filter = dict(
    data_vis=['B02', 'B03', 'B04', 'B08'],   # Red, Green, Blue, NIR
    data_source=['labels', 'source_sentinel_2']
   
)

In [91]:
my_geojson = json.loads(bb)

In [92]:
dataset.download(collection_filter=my_filter, intersects=my_geojson)

#dataset.download(collection_filter=my_filter)


INFO:radiant_mlhub.client.catalog_downloader:unarchive ref_landcovernet_af_v1.tar.gz ...
unarchive ref_landcovernet_af_v1.tar.gz: 100%|██████████| 603550/603550 [00:33<00:00, 17842.33it/s]
INFO:radiant_mlhub.client.catalog_downloader:create stac asset list (please wait) ...
INFO:radiant_mlhub.client.catalog_downloader:2731368 unique assets in stac catalog.
INFO:radiant_mlhub.client.catalog_downloader:filter by collection ids and asset keys


RuntimeError: after filtering collections_ids and asset keys, zero assets to download. filter: <class 'filter'>